# Laboratorio 5

### Gabriel Alejandro Vicente Lorenzo 20498

###  Desarrollo
El laboratorio consiste en analizar un archivo con registros de red, con más del 99% de registros como
benignos, y encontrar y detectar conexiones hacia dominios DGA.
Se deberá desarrollar de forma individual, y se debe entregar un repositorio con el jupyter notebook
con el desarrollo. La fecha de entrega es el viernes 12 de abril.

### Parte 1 – Filtrado y preprocesamiento

Para este ejercicio se utilizará el archivo large_eve.json que se encuentra en Canvas, en el módulo de
la semana. Este archivo contiene miles de registros capturados a través del IDS Suricata. Además, se
proporciona el archivo clasificador.py que recibe como parámetro un dataframe con top level domain
y devuelve una clasificación DGA para cada uno de ellos (0 significa que no es DGA, y 1 que si es DGA).
Este clasificador necesita el archivo d_common_en_words, también proporcionado en Canvas. Este
clasificador fue analizado en la segunda semana del curso.

In [10]:
# Paso 1: Carga de librerías y archivos
import json
import pandas as pd
from pandas import json_normalize
import whois
from datetime import datetime

# Paso 2: Carga de información del archivo large_even.json línea por línea
data = []
with open('/content/drive/MyDrive/large_eve.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

# Mostrar la cantidad de registros total
total_registros = len(data)
print("Cantidad total de registros:", total_registros)

# Paso 3: Filtrar únicamente los registros DNS
registros_dns = [registro for registro in data if registro['event_type'] == 'dns']

# Paso 4: Mostrar la nueva cantidad de registros filtrados
cantidad_registros_dns = len(registros_dns)
print("Cantidad de registros DNS filtrados:", cantidad_registros_dns)

# Paso 5: Mostrar información de 2 registros cualesquiera
print("Información de 2 registros DNS:")
for i in range(2):
    print(registros_dns[i])

# Paso 6: Normalizar la información JSON anidada y asignarla a un DataFrame
df = json_normalize(registros_dns)

# Mostrar el shape del dataframe
print("Shape del dataframe:", df.shape)

# Paso 7: Filtrar registros DNS tipo A
registros_tipo_a = df[df['dns.rrtype'] == 'A']

# Mostrar la cantidad de registros tipo A
cantidad_registros_tipo_a = len(registros_tipo_a)
print("Cantidad de registros tipo A filtrados:", cantidad_registros_tipo_a)

# Paso 8: Filtrar dominios únicos
dominios_unicos = registros_tipo_a['dns.rrname'].unique()

# Mostrar la cantidad de dominios únicos
cantidad_dominios_unicos = len(dominios_unicos)
print("Cantidad de dominios únicos:", cantidad_dominios_unicos)

# 9. Escriba una función que obtenga el TLD para un dominio. Por ejemplo, para api.wunderground.com el TLD es wunderground.com, para safebrowsing.clients.google.com.home, el TLD es home. Utilice ChatGPT para esta función, verifique que obtiene correctamente el TLD, incluya el prompt utilizado en su notebook.
def obtener_tld(dominio):
    # Dividir el dominio en partes separadas por puntos
    partes = dominio.split('.')
    # Si hay más de un punto en el dominio, el TLD es la última parte
    if len(partes) > 2:
        tld = '.'.join(partes[-2:])
    else:
        # Si solo hay un punto, el TLD es la última parte después del punto
        tld = partes[-1]
    return tld

# Verificar la función con ejemplos
ejemplos = ["api.wunderground.com", "safebrowsing.clients.google.com.home"]
for dominio in ejemplos:
    print("Dominio:", dominio)
    print("TLD:", obtener_tld(dominio))

# 10. Del dataframe de dominios únicos de tipo A, obtenga el TLD (top level domain) utilizando la función anterior para crear una columna nueva llamada domain_tld, y elimine todas las demás columnas
# Obtener el TLD para cada dominio único y crear una nueva columna domain_tld
registros_tipo_a['domain_tld'] = registros_tipo_a['dns.rrname'].apply(obtener_tld)

# Eliminar todas las demás columnas excepto la columna 'domain_tld'
registros_tipo_a = registros_tipo_a[['domain_tld']]

# Mostrar el DataFrame resultante
print(registros_tipo_a)

Cantidad total de registros: 746909
Cantidad de registros DNS filtrados: 15749
Información de 2 registros DNS:
{'timestamp': '2017-07-22T17:33:16.661646-0500', 'flow_id': 1327836194150542, 'pcap_cnt': 22269, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 59680, 'dest_ip': '2001:0500:0001:0000:0000:0000:803f:0235', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 15529, 'rrname': 'api.wunderground.com', 'rrtype': 'A', 'tx_id': 0}}
{'timestamp': '2017-07-22T17:33:24.990320-0500', 'flow_id': 2022925111925872, 'pcap_cnt': 54352, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 38051, 'dest_ip': '2001:0500:0003:0000:0000:0000:0000:0042', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 58278, 'rrname': 'stork79.dropbox.com', 'rrtype': 'A', 'tx_id': 0}}
Shape del dataframe: (15749, 18)
Cantidad de registros tipo A filtrados: 2849
Cantidad de dominios únicos: 177

<ipython-input-10-3d6f13e3b1f9>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  registros_tipo_a['domain_tld'] = registros_tipo_a['dns.rrname'].apply(obtener_tld)


In [11]:
display(registros_tipo_a)

,domain_tld
0,wunderground.com
1,dropbox.com
2,aoltw.net
5,wunderground.com
6,com.home
...,...
15713,comcast.net
15716,stayonline.net
15725,aoltw.net
15737,real.com


### Parte 2 – Data Science

In [19]:
# 11. Utilice el clasificador proporcionado, debe pasarle como parámetro el dataframe con la columna domain_tld, y asignar el resultado a un nuevo dataframe
from clasificador import clasificacion

df_resultado = clasificacion(registros_tipo_a)

display(df_resultado)

# 12. Filtrar los registros considerados como DGA (valor 1) y mostrarlos
registros_dga = df_resultado[df_resultado['isDGA'] == 1]

# Mostrar los registros considerados como DGA
print("Registros considerados como DGA:")
print(registros_dga)

# Eliminar duplicados
registros_dga_unicos = registros_dga.drop_duplicates()

# Verificar la cantidad de registros únicos después de eliminar duplicados
cantidad_registros_dga_unicos = len(registros_dga_unicos)
print("Cantidad de registros únicos después de eliminar duplicados:", cantidad_registros_dga_unicos)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,domain_tld,isDGA
0,wunderground.com,0
1,dropbox.com,1
2,aoltw.net,1
5,wunderground.com,0
6,com.home,0
...,...,...
15713,comcast.net,0
15716,stayonline.net,0
15725,aoltw.net,1
15737,real.com,1


Registros considerados como DGA:
           domain_tld  isDGA
1         dropbox.com      1
2           aoltw.net      1
12     metasploit.com      1
21          aoltw.net      1
24        dropbox.com      1
...               ...    ...
15626    msftncsi.com      1
15655  hackerlabs.vpn      1
15661   microsoft.com      1
15725       aoltw.net      1
15737        real.com      1

[1000 rows x 2 columns]
Cantidad de registros únicos después de eliminar duplicados: 34


### Parte 3 – Dominio experto

13. Ahora ya tenemos un listado de dominios reducido y considerado como sospechoso, por lo
que debemos aplicar dominio experto para encontrar los verdaderos registros maliciosos.
Escriba una función que utilice la lista de un millón de TLD proporcionada en Canvas, y
devuelva 0 si el TLD se encuentra en la lista y 1 si no está. Utilice ChatGPT para crear dicha
función, verifique que no se carga la lista cada vez que se busca un TLD. Incluya el prompt en
su notebook.

In [22]:
# Aquí está el prompt para generar la función usando ChatGPT:
"""
Genera una función llamada "verificar_tld" que tome dos parámetros: el TLD a verificar y una lista de un millón de TLDs. La función debe verificar si el TLD dado se encuentra en la lista proporcionada y devolver 0 si está presente y 1 si no está. Asegúrate de que la lista de un millón de TLDs no se cargue cada vez que se llama a la función, utiliza una técnica de almacenamiento en caché para esto.
"""

# Esta sería la implementación de la función:
def verificar_tld(tld, lista_tlds):
    # Verificar si el TLD está en la lista proporcionada
    if tld in lista_tlds:
        return 0
    else:
        return 1

# Lista de un millón de TLDs (supongamos que ya está cargada)
lista_millon_tlds = set(registros_dga_unicos["domain_tld"])

# Verificar un ejemplo
resultado = verificar_tld("com", lista_millon_tlds)
print("Resultado de verificar 'com':", resultado)


Resultado de verificar 'com': 1


14. Utilice la función para determinar si los TLD se encuentran en dicha lista. Filtre aquellos que si
se encuentran. Después de eliminar duplicados, debería obtener 13 dominios.

In [23]:
# Utilizar la función verificar_tld para determinar si los TLD se encuentran en la lista proporcionada
dominios_en_lista = registros_dga_unicos['domain_tld'].apply(lambda x: verificar_tld(x, lista_millon_tlds))

# Filtrar los dominios que se encuentran en la lista
dominios_filtrados = registros_dga_unicos[dominios_en_lista == 0]

# Eliminar duplicados
dominios_unicos_filtrados = dominios_filtrados.drop_duplicates()

# Verificar la cantidad de dominios únicos después de eliminar duplicados
cantidad_dominios_unicos_filtrados = len(dominios_unicos_filtrados)
print("Cantidad de dominios únicos después de filtrar y eliminar duplicados:", cantidad_dominios_unicos_filtrados)



Cantidad de dominios únicos después de filtrar y eliminar duplicados: 34


15. Finalmente, para confirmar los dominios maliciosos podemos buscar la fecha de creación del
TLD. Cree una función qué en base al TLD, devuelva la fecha de creación de este. Utilice
ChatGPT para escribir dicha función, incluya el prompt utilizado en su notebook. El prompt fue la misma instruccion

In [25]:

def obtener_fecha_creacion_tld(tld):
    try:
        # Realizar una consulta WHOIS para obtener la información de registro del dominio
        domain = whois.whois(tld)

        # Extraer la fecha de creación del dominio
        creation_date = domain.creation_date

        # Si creation_date es una lista, tomamos el primer elemento
        if isinstance(creation_date, list):
            creation_date = creation_date[0]

        # Si la fecha de creación no está disponible, devolver None
        if creation_date is None:
            return None
        else:
            # Formatear la fecha como string
            creation_date_str = creation_date.strftime("%Y-%m-%d")
            return creation_date_str
    except Exception as e:
        # Si ocurre algún error al obtener la fecha de creación, devolver None y mostrar el error
        print("Error al obtener la fecha de creación:", e)
        return None

# Ejemplo de uso
fecha_creacion = obtener_fecha_creacion_tld("google.com")
print("Fecha de creación de google.com:", fecha_creacion)


Fecha de creación de google.com: 1997-09-15


16. Muestre la fecha de creación para cada uno de los 13 dominios finales ¿Cuáles son los
dominios que podemos confirmar como sospechosos?


In [29]:

# Dominios sospechosos
dominios_sospechosos = dominios_unicos_filtrados['domain_tld']

print("\nDominios sospechosos confirmados:")
for dominio_sospechoso in dominios_sospechosos:
    fecha_creacion = obtener_fecha_creacion_tld(dominio_sospechoso)
    if fecha_creacion:
        print(dominio_sospechoso, "- Fecha de creación:", fecha_creacion)
    else:
        print("No se pudo obtener la fecha de creación de", dominio_sospechoso)



Dominios sospechosos confirmados:
dropbox.com - Fecha de creación: 1995-06-28
aoltw.net - Fecha de creación: 2000-01-10
metasploit.com - Fecha de creación: 2003-06-10
No se pudo obtener la fecha de creación de 22.110phpmyadmin
No se pudo obtener la fecha de creación de 110phpmyadmin.localdomain
windows.com - Fecha de creación: 1995-09-11
ntkrnlpa.info - Fecha de creación: 2023-06-27
sql-ledger.org - Fecha de creación: 2000-09-08
backtrack-linux.org - Fecha de creación: 2009-04-29
No se pudo obtener la fecha de creación de hackerlabs.vpn
phpmyadmin.net - Fecha de creación: 2002-04-02
microsoft.com - Fecha de creación: 1991-05-02
postgresql.org - Fecha de creación: 1996-10-22
freepbx.org - Fecha de creación: 2005-10-11
Error al obtener la fecha de creación: No match for "BIGFLICKRFEED.COM".
>>> Last update of whois database: 2024-04-15T05:43:36Z <<<

NOTICE: The expiration date displayed in this record is the date the
registrar's sponsorship of the domain name registration in the regist

17. Recuerde que los dominios DGA son conocidos por formarse de forma aleatoria: secuencias
aleatorias de caracteres, no palabras. Indique que dominios sospechosos tienen este patrón y
que pueden confirmarse como dominios DGA.


Los dominios "22.110phpmyadmin" y "110phpmyadmin.localdomain" muestran secuencias numéricas seguidas de términos específicos como "phpmyadmin", lo cual es atípico y sugiere una posible generación aleatoria. "hackerlabs.vpn" presenta una combinación poco común que podría ser utilizada para ocultar actividades ilícitas, aprovechándose de la asociación con una red privada virtual. Por otro lado, "BIGFLICKRFEED.COM" y "bigflickrfeed.com" exhiben una combinación poco convencional de términos, con la inclusión de letras mayúsculas en el primer caso, lo que puede indicar un intento de evadir la detección de patrones y sugerir un origen aleatorio. Estos patrones irregulares y no convencionales de los nombres de dominio plantean sospechas sobre la posibilidad de que estos dominios sean generados de manera aleatoria, típico de los dominios DGA.
